**Importing  Requried Libraries**

In [1]:
#!pip install pytesseract
#!apt install tesseract-ocr

In [2]:
import cv2
from PIL import Image
import pytesseract as tess
import numpy as np


In [3]:
#!apt autoremove
#!apt update

**Reading And  Displaying The Image**

In [4]:
img = cv2.imread("VW_Jetta_Front.jpg")



In [5]:
img = cv2.resize(img,(620,480))
cv2.imshow("original image",img); cv2.waitKey(0); cv2.destroyAllWindows()


**Image Pre-Prosseing**

In [6]:
imgBlurred = cv2.GaussianBlur(img, (3,3), 0) # removing gaussian noise from the image
gray = cv2.cvtColor(imgBlurred, cv2.COLOR_BGR2GRAY) # converting BGR image to Grey


#gray = cv2.adaptiveThreshold(gray,50,cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY,11,2)

gray = cv2.Canny(gray,150,255)
cv2.imshow("Canny_Filtered",gray); cv2.waitKey(0); cv2.destroyAllWindows()

#sobelx = cv2.GaussianBlur(sobelx,(5,3),0)
sobelx = cv2.Sobel(gray,cv2.CV_8U,1,0,ksize=3) # applying sobelX Filter


	
ret2,threshold_img = cv2.threshold(sobelx,210,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
#cv2_imshow(threshold_img)

cv2.imshow("SObelX_threshold",threshold_img); cv2.waitKey(0); cv2.destroyAllWindows()

**Contours Extraction**

In [7]:
element = cv2.getStructuringElement(shape=cv2.MORPH_RECT, ksize=(17, 3))
morph_img_threshold = threshold_img.copy()
cv2.morphologyEx(src=threshold_img, op=cv2.MORPH_CLOSE, kernel=element, dst=morph_img_threshold)
contours,_ = cv2.findContours(morph_img_threshold,mode=cv2.RETR_EXTERNAL,method=cv2.CHAIN_APPROX_NONE)

**Draw Contours On The Image**

In [8]:
img_copy = img.copy()

In [9]:
cv2.drawContours(img_copy, contours, -1, (0,255,0), 1)
cv2.imshow("Contours on image",img_copy); cv2.waitKey(0); cv2.destroyAllWindows()


**Filterning the Unwated Contours from the image**

In [10]:
def validateRotationAndRatio(rect):
	(x, y), (width, height), rect_angle = rect

	if(width>height):
		angle = -rect_angle
	else:
		angle = 90 + rect_angle

	if angle>15:
	 	return False

	if height == 0 or width == 0:
		return False

	area = height*width
	

	return True

In [11]:
#!apt install libtesseract-dev

def ratioCheck(area, width, height):
	ratio = float(width) / float(height)
	if ratio < 1:
		ratio = 1 / ratio

	aspect = 4.7272
	min = 15*aspect*15  # minimum area
	max = 125*aspect*125  # maximum area

	rmin = 3
	rmax = 6

	if (area < min or area > max) or (ratio < rmin or ratio > rmax):
		return False
	return True

In [12]:
def isPlate(plate_img):
  
    char = cv2.cvtColor(plate_img, cv2.COLOR_BGR2GRAY)
    #char = cv2.Canny(char,120,255)

    (_, thresh) = cv2.threshold(char, 200, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    #thresh = cv2.adaptiveThreshold(char,255,cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY,2,1)
    #cv2_imshow(thresh)

    contours,hierarchy = cv2.findContours(thresh.copy(),cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    
    if contours:
        areas = [cv2.contourArea(c) for c in contours]
        #print("areas=={}",areas)
        max_index = np.argmax(areas)
        #print("max_index",max_index)
        max_cnt = contours[max_index]
        #print("contours(max_cnt)=",max_cnt)
        max_cntArea = areas[max_index]
        #print("areas[max_index]=",max_cntArea)
        x,y,w,h = cv2.boundingRect(max_cnt)
        
        if not ratioCheck(max_cntArea,w,h):
            return plate_img,None
        
        cleaned_final = thresh[y:y+h, x:x+w]
        return cleaned_final,[x,y,w,h]
      
    else:
          return plate,None
        
  

In [13]:
for i,cnt in enumerate(contours):
    min_rect = cv2.minAreaRect(cnt)
    if validateRotationAndRatio(min_rect):
        x,y,w,h = cv2.boundingRect(cnt)
        plate_img = img[y:y+h,x:x+w]
        #cv2_imshow(plate_img)
        #plate_img = cv2.adaptiveThreshold(plate_img,255,cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY,11,2)

        numPlate, rect = isPlate(plate_img)


        if rect:
            print("NumberPlate")
            #numPlate = cv2.Sobel(numPlate,cv2.CV_8U,1,0,ksize=3)
            #numPlate = cv2.Canny(numPlate,200,255)
            cv2.imshow("RECT boxes_probable number plate",numPlate);cv2.waitKey(0); cv2.destroyAllWindows()
            print(tess.image_to_string(numPlate))
            x1,y1,w1,h1 = rect
            x,y,w,h = x+x1,y+y1,w1,h1
            plateonly = img[y:y+h,x:x+w]
            


          

NumberPlate
MH 14087483:
